In [ ]:
'''import os
import zipfile
local_zip = 'real-vs-fake.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('')
zip_ref.close()'''

In [2]:
%pip install scikit-learn --upgrade
import numpy as np
import pandas as pd 
import matplotlib.pyplot as pl 
import os 
from tensorflow.keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, regularizers

Note: you may need to restart the kernel to use updated packages.


2025-05-01 18:41:19.497994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746124880.472994    3603 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746124880.763810    3603 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746124883.431029    3603 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746124883.431052    3603 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746124883.431055    3603 computation_placer.cc:177] computation placer alr

In [3]:
img_rows, img_cols = 224 , 224
batch_size = 32 
SEED = 42 
num_classes = 2 
train_data_dir = 'real-vs-fake/train'
validation_data_dir = 'real-vs-fake/valida'
test_data_dir = 'real-vs-fake/test'
labels = ['Real', 'Fake']


In [4]:
import tensorflow as tf

# For training data
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_data_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(224, 224),
    batch_size=batch_size,
    seed=SEED,
    shuffle=True
).map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)).prefetch(tf.data.AUTOTUNE)

# For validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
    validation_data_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(224, 224),
    batch_size=batch_size,
    seed=SEED,
    shuffle=True
).map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)).prefetch(tf.data.AUTOTUNE)

# For test data
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_data_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(224, 224),
    batch_size=batch_size,
    shuffle=False
).map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)).prefetch(tf.data.AUTOTUNE)


Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


I0000 00:00:1746124913.401245    3603 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79088 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0001:00:00.0, compute capability: 8.0


In [5]:
from tensorflow.keras.applications import InceptionResNetV2
base_model = InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(img_rows, img_cols, 3),
    pooling='avg'
)
base_model.trainable = False





219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [19]:
from keras import layers, models, regularizers
from tensorflow.keras import Model, Input
from tensorflow.keras.applications import InceptionResNetV2

input_tensor = Input(shape=(224, 224, 3))
base_model = InceptionResNetV2(include_top=False, weights='imagenet', pooling='avg', input_tensor=input_tensor)
base_model.trainable = False

x = base_model.output
x = layers.Dense(512, kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.LeakyReLU(alpha=0.2)(x)  # use as a separate layer
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_tensor, outputs=output)



/anaconda/envs/azureml_py38_PT_TF/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [20]:
import keras
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, LearningRateScheduler



def exp_decay(epoch):
    initial_lrate = 1e-5
    k = 0.1
    return initial_lrate * np.exp(-k * epoch)

class LossHistory_(Callback):
    def on_train_begin(self, logs=None):
        self.losses = []
        self.lr = []

    def on_epoch_end(self, epoch, logs=None):
        current_lr = exp_decay(epoch)
        self.losses.append(logs.get('loss'))
        self.lr.append(current_lr)
        print(f"Epoch {epoch+1} - Learning Rate: {current_lr:.6f}")



In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [22]:
loss_history_ = LossHistory_()
lrate_ = LearningRateScheduler(exp_decay)

keras_callbacks = [
    EarlyStopping(monitor='loss', patience=5, mode='min', min_delta=0.01),
    ModelCheckpoint(checkpoint_path, monitor='loss', save_best_only=True, mode='min')
]

callbacks_list = [loss_history_, lrate_] + keras_callbacks

In [12]:
model.fit(train_ds, validation_data=val_ds, epochs=10, verbose=1, callbacks=callbacks_list)


Epoch 1/10
3123/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6939 - loss: 5.8418Epoch 1 - Learning Rate: 0.000010
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 119s 38ms/step - accuracy: 0.6939 - loss: 5.8415 - val_accuracy: 0.7700 - val_loss: 4.7934 - learning_rate: 1.0000e-05
Epoch 2/10
3124/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7248 - loss: 4.6702Epoch 2 - Learning Rate: 0.000009
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 117s 37ms/step - accuracy: 0.7248 - loss: 4.6701 - val_accuracy: 0.7872 - val_loss: 3.9821 - learning_rate: 9.0484e-06
Epoch 3/10
3124/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7400 - loss: 3.9238Epoch 3 - Learning Rate: 0.000008
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 106s 34ms/step - accuracy: 0.7400 - loss: 3.9237 - val_accuracy: 0.7963 - val_loss: 3.4266 - learning_rate: 8.1873e-06
Epoch 4/10
3124/3125 ━━━━━━━━━━━━━━━━━━━

In [23]:
for layer in base_model.layers[:450]:
        layer.trainable = False

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-6),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [25]:
model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=callbacks_list)

Epoch 1/5
3124/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6267 - loss: 7.7850Epoch 1 - Learning Rate: 0.000010
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 133s 37ms/step - accuracy: 0.6267 - loss: 7.7845 - val_accuracy: 0.7509 - val_loss: 5.9306 - learning_rate: 1.0000e-05
Epoch 2/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6951 - loss: 5.7220Epoch 2 - Learning Rate: 0.000009
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 107s 34ms/step - accuracy: 0.6951 - loss: 5.7219 - val_accuracy: 0.7764 - val_loss: 4.7061 - learning_rate: 9.0484e-06
Epoch 3/5
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7186 - loss: 4.6285Epoch 3 - Learning Rate: 0.000008
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 108s 35ms/step - accuracy: 0.7186 - loss: 4.6285 - val_accuracy: 0.7890 - val_loss: 3.9630 - learning_rate: 8.1873e-06
Epoch 4/5
3123/3125 ━━━━━━━━━━━━━━━━━━━━ 0s

In [26]:
model.save("final_inceptionresnetv2_deepfake_model_74.h5")

In [5]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras import Model, Input, layers, regularizers
from tensorflow.keras.optimizers import Adam


base_model = InceptionResNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
      
)



In [6]:
for layer in base_model.layers[:450]:
        layer.trainable = False

    


In [7]:
from tensorflow.keras import models

model = models.Sequential([
        base_model,                               
        layers.GlobalAveragePooling2D(),         
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  
        layers.BatchNormalization(),            
        layers.Dropout(0.5),                     
        layers.Dense(1, activation='sigmoid')    
    ])



In [8]:
 # Compile the model
model.compile(
        optimizer=Adam(learning_rate=0.0001),  
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

In [10]:


# Stop if the validation loss is not improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model
history_fine_tune = model.fit(
    train_ds,             
    validation_data=val_ds,  
    epochs=10,                  
    callbacks=[early_stopping]
)

Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 176s 56ms/step - accuracy: 0.8618 - loss: 4.4749 - val_accuracy: 0.9718 - val_loss: 0.1884
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 169s 54ms/step - accuracy: 0.9730 - loss: 0.1191 - val_accuracy: 0.9822 - val_loss: 0.0623
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 167s 53ms/step - accuracy: 0.9865 - loss: 0.0461 - val_accuracy: 0.9753 - val_loss: 0.0786
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 165s 53ms/step - accuracy: 0.9926 - loss: 0.0260 - val_accuracy: 0.9815 - val_loss: 0.0613
Epoch 5/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 164s 53ms/step - accuracy: 0.9930 - loss: 0.0269 - val_accuracy: 0.9814 - val_loss: 0.0586
Epoch 6/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 163s 52ms/step - accuracy: 0.9954 - loss: 0.0170 - val_accuracy: 0.9722 - val_loss: 0.0910

In [13]:
model.save("final_inceptionresnetv2_deepfake_model_99_new.h5")

In [12]:
loss, accuracy = model.evaluate(test_ds, verbose=1)

# Print the results
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


625/625 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.9918 - loss: 0.0254
Test Loss: 0.0292
Test Accuracy: 0.9909
